# Convolution Variational Autoencoder

Autoencoder helps you to input some image and map it some latent vector for decoder to a new image.
Variational Autoencoder on the other hand help you to map the encoder output to some mean and sigma from which we map our latent dimmension as a result we get some stochasticity in z.So latent dimension is not totaly dependent on im=nput image for which we can tweak the network easly to make new changes and create new image.

# Import of libraries

In [0]:
!pip install -q imageio

In [0]:
import tensorflow as tf

import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio

from IPython import display


# Load the mnist Dataset

In [0]:
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()


In [0]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
train_images=train_images/255.0
test_images=test_images/255.0

"""binarisation
train_images[train_images >= .5] = 1.
train_images[train_images < .5] = 0.
test_images[test_images >= .5] = 1.
test_images[test_images < .5] = 0."""


In [0]:
for i in range(4):
  plt.subplot(2,2,i+1)
  plt.imshow(train_images[i,:,:,0],cmap="gray")

In [0]:
TRAIN_BUF = 60000
BATCH_SIZE = 1000

TEST_BUF = 10000


In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(TRAIN_BUF).batch(BATCH_SIZE).prefetch(1)
test_dataset = tf.data.Dataset.from_tensor_slices(test_images).shuffle(TEST_BUF).batch(BATCH_SIZE).prefetch(1)


# Model

It contains a encoder which maps image ->mean,logvariance<br>
A reparametarize that ads noise mean+(logvar->sigmoid)*noise<br>
A generator for generation of latent vector to new image

In [0]:
class VAE(tf.keras.Model):
  def __init__(self,latent_dim):
    super(VAE,self).__init__()
    initial=tf.random_normal_initializer(0.0,0.02)
    self.latent_dim=latent_dim
    self.autoencoder_net=tf.keras.Sequential(
        [tf.keras.layers.InputLayer(input_shape=(28,28,1)),
         tf.keras.layers.Conv2D(32,3,(1,1),"same",activation="relu"),
         tf.keras.layers.Conv2D(64,3,(2,2),"same",activation="relu"),
         tf.keras.layers.Conv2D(64,3,(1,1),"same",activation="relu"),
         tf.keras.layers.Conv2D(64,3,(2,2),"same",use_bias=True,activation="relu"),
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dense(128,activation="relu")
        ]
    )
    self.means=tf.keras.layers.Dense(units=self.latent_dim)
    self.logvars=tf.keras.layers.Dense(units=self.latent_dim)
    self.generative_net=tf.keras.Sequential(
        [tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
         tf.keras.layers.Dense(units=7*7*128,activation="relu"),
         tf.keras.layers.Reshape((7,7,128)),
         tf.keras.layers.Conv2DTranspose(64,3,(2,2),"same",activation="relu",use_bias=False),
         tf.keras.layers.Conv2DTranspose(32,3,(2,2),"same",activation="relu",use_bias=False),
         tf.keras.layers.Conv2DTranspose(1,3,(1,1),"same",activation="tanh",use_bias=False)]
    )
  def encode(self,x):
    z=self.autoencoder_net(x)
    mean=self.means(z)
    logvariance=self.logvars(z)
    return mean,logvariance
  
  def reparametarize(self,mean,logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * 0.5) + mean#epsilon*sigma+mean
  
  def decode(self,z,apply_sigmoid=False):
    y=self.generative_net(z)
    if apply_sigmoid:
      y=tf.sigmoid(y)#only during testing for cliping values from [-1,1] to [0,1]
    return y
  
  def sample_noise(self,example):
    return self.decode(noise,apply_sigmoid=False)
  
  def sample_image(self,image_ex):
    u,lnv=self.encode(image_ex)
    noise_z=self.reparametarize(u,lnv)
    return self.decode(noise_z,apply_sigmoid=False)
    



#Now the loss function

The loss function

<img src="/content/vae.JPG">

In [0]:
optimizer=tf.keras.optimizers.Adam(1e-4)

In [0]:
@tf.function
def vae_loss(model,x):
  mean,logvar=model.encode(x)
  z=model.reparametarize(mean,logvar)
  y=model.decode(z)
  logpx_z =tf.reduce_sum(tf.keras.losses.MSE(x,y),axis=(1,2))
  kldivergence=-0.5*(tf.reduce_sum((1+logvar-tf.square(mean)-tf.exp(logvar)),axis=1))
  total_loss=tf.reduce_mean(logpx_z+kldivergence)
  return logpx_z,total_loss



In [0]:
@tf.function
def vae_loss1(model,x):
  mean,logvar=model.encode(x)
  z=model.reparametarize(mean,logvar)
  y=model.decode(z)
  marginal_likelihood = tf.reduce_sum(x * tf.math.log(y) + (1 - x) * tf.math.log(1 - y), [1, 2])
  KL_divergence = 0.5 * tf.reduce_sum(tf.square(mean) + tf.exp(logvar) - logvar - 1, [1])
  neg_loglikelihood = -tf.reduce_mean(marginal_likelihood)
  KL_divergence = tf.reduce_mean(KL_divergence)
  total_loss=-neg_loglikelihood-KL_divergence
  total_loss=-total_loss
  return total_loss



In [0]:
@tf.function
def train_step(model,x):
  with tf.GradientTape() as tape:
    loss=vae_loss1(model,x)
  gradients=tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  return loss


# Generate Images

In [0]:
def generate_noise_save_images(model,epoch,num_of_examples):
  predictions=model.sample_noise(num_of_examples)
  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0], cmap='gray')
      plt.axis('off')

  # tight_layout minimizes the overlap between 2 sub-plots
  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()
  #print(predictions[0])


# Training

In [0]:
EPOCHS=100
latent_dim = 100
num_examples_to_generate = 16


In [0]:
noise=tf.random.normal(shape=(16,latent_dim))

In [0]:
model=VAE(latent_dim)

In [0]:
generate_noise_save_images(model, 0, noise)

for epoch in range(1, EPOCHS + 1):
  start_time = time.time()
  mean_loss=[]
  for train_x in train_dataset:
    loss=train_step(model, train_x)
    mean_loss.append(loss)
    #print(bh,bhh,bruh)
  end_time = time.time()
  print('Epoch: {}, LOSS: {}, '
          'time elapse for current epoch {}'.format(epoch,
                                                    np.mean(np.asarray(mean_loss)),
                                                    end_time - start_time))
  generate_noise_save_images(
        model, epoch,noise)


In [0]:
!rm *.png

In [0]:
plt.plot(mean_loss)#latent dim 50

In [0]:
plt.plot(mean_loss)#latent dim 2

In [0]:
anim_file = 'cvae3.gif'

with imageio.get_writer(anim_file,fps=5, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

import IPython
if IPython.version_info >= (6,2,0,''):
  display.Image(filename=anim_file)
